Replication of this [kaggle notebook](https://www.kaggle.com/janniskueck/pm3-notebook-inference-clustering)

In [5]:
# Import relevant packages
import pandas as pd
import numpy as np
import pyreadr
from sklearn import preprocessing
import patsy

from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

import hdmpy
import numpy as np
import random
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors

In [6]:
#!pip install tensorflow

In [7]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import itertools
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_categorical_dtype
from itertools import compress
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.feature_selection import SelectFromModel

This notebook contains an example for teaching.

# A Case Study: The Effect of Gun Ownership on Gun-Homicide Rates

We consider the problem of estimating the effect of gun
ownership on the homicide rate. For this purpose, we estimate the following partially
linear model

$$
 Y_{j,t} = \beta D_{j,(t-1)} + g(Z_{j,t}) + \epsilon_{j,t}.
$$

## Data

$Y_{j,t}$ is log homicide rate in county $j$ at time $t$, $D_{j, t-1}$ is log  fraction of suicides committed with a firearm in county $j$ at time $t-1$, which we use as a proxy for gun ownership,  and  $Z_{j,t}$ is a set of demographic and economic characteristics of county $j$ at time $t$. The parameter $\beta$ is the effect of gun ownership on the
homicide rates, controlling for county-level demographic and economic characteristics. 

The sample covers 195 large United States counties between the years 1980 through 1999, giving us 3900 observations.

In [19]:
data1 = pd.read_csv( r"../../../data/gun_clean.csv" )
data1.shape[1]

415

In [20]:
# load dataset
#rdata_read = pyreadr.read_r("../../../data/gun_clean.RData")
#data = rdata_read[ 'data' ]
#n = data.shape[1]

In [21]:
n

198

### Preprocessing

To account for heterogeneity across counties and time trends in  all variables, we remove from them county-specific and time-specific effects in the following preprocessing.

In [9]:
import re

In [10]:
#################################  Find Variable Names from Dataset ########################

def varlist( df = None, type_dataframe = [ 'numeric' , 'categorical' , 'string' ],  pattern = "", exclude = None ):
    varrs = []
    
    if ('numeric' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_numeric_dtype , axis = 0 ).to_list() ].tolist()
    
    if ('categorical' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_categorical_dtype , axis = 0 ).to_list() ].tolist()
    
    if ('string' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_string_dtype , axis = 0 ).to_list() ].tolist()
    
    grepl_result = np.array( [ re.search( pattern , variable ) is not None for variable in df.columns.to_list() ] )
    
    if exclude is None:
        result = list(compress( varrs, grepl_result ) )
    
    else:
        varrs_excluded = np.array( [var in exclude for var in varrs ] )
        and_filter = np.logical_and( ~varrs_excluded ,  grepl_result )
        result = list(compress( varrs, and_filter ) )
    
    return result   

################################# Create Variables ###############################


# Dummy Variables for Year and County Fixed Effects
r = re.compile("X_Jfips")
fixed = list( filter( r.match, data1.columns.to_list() ) )
year = varlist(data1, pattern="X_Tyear")

census = []
census_var = ["^AGE", "^BN", "^BP", "^BZ", "^ED", "^EL","^HI", "^HS", "^INC", "^LF", "^LN", "^PI", "^PO", "^PP", "^PV", "^SPR", "^VS"]

for variable in census_var:
    r = re.compile( variable )
    census = census + list( filter( r.match, data1.columns.to_list() ) )

    
################################ Variables ##################################
# Treatment Variable
d = "logfssl"

# Outcome Variable
y = "logghomr"

# Other Control Variables
X1 = ["logrobr", "logburg", "burg_missing", "robrate_missing"]
X2 = ["newblack", "newfhh", "newmove", "newdens", "newmal"]

#################################  Partial out Fixed Effects ########################

rdata = data1[['CountyCode']]

# Variables to be Partialled-out
varlist2 = [y] + [d] + X1 + X2 + census

# Partial out Variables in varlist from year and county fixed effect
for var_name in varlist2:
    form = var_name + " ~ "  + " + ".join( year ) + "+" + " + ".join( fixed )
    rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid

C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a cop

C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a cop

C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a cop

C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a cop

C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a cop

...

C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the

C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the

C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the

C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the

C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata[f'{var_name}'] = smf.ols( formula = form , data = data1 ).fit().resid
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\3290486364.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the

## We check that our results are equal to R results at 6 decimals

In [22]:
column_names = data.columns.to_list()

for col in column_names:
    result = (data[f'{col}'].round(6) == rdata[f'{col}'].round(6)).sum()
    
    if result == 3900:
        print(f'Column {col} are equal at 6 decimals.')
    else:
        print(f'\n\nColumn {col} are not equal at 6 decimals.')

Column logghomr are equal at 6 decimals.
Column logfssl are equal at 6 decimals.
Column logrobr are equal at 6 decimals.
Column logburg are equal at 6 decimals.
Column burg_missing are equal at 6 decimals.
Column robrate_missing are equal at 6 decimals.
Column newblack are equal at 6 decimals.
Column newfhh are equal at 6 decimals.
Column newmove are equal at 6 decimals.
Column newdens are equal at 6 decimals.
Column newmal are equal at 6 decimals.
Column AGE010D are equal at 6 decimals.
Column AGE050D are equal at 6 decimals.
Column AGE110D are equal at 6 decimals.
Column AGE170D are equal at 6 decimals.
Column AGE180D are equal at 6 decimals.
Column AGE270D are equal at 6 decimals.
Column AGE310D are equal at 6 decimals.
Column AGE320D are equal at 6 decimals.
Column AGE350D are equal at 6 decimals.
Column AGE380D are equal at 6 decimals.
Column AGE410D are equal at 6 decimals.
Column AGE470D are equal at 6 decimals.
Column AGE570D are equal at 6 decimals.
Column AGE640D are equal at

Now, we can construct the treatment variable, the outcome variable and the matrix $Z$ that includes the control variables.

In [23]:
# Treatment Variable
D = rdata[ f'{d}']

# Outcome Variable
Y = rdata[ f'{y}']

# Construct matrix Z
Z = rdata[ X1 + X2 + census ]

Z.shape

(3900, 195)

We have in total 195 control variables. The control variables $Z_{j,t}$ are from the U.S. Census Bureau and  contain demographic and economic characteristics of the counties such as  the age distribution, the income distribution, crime rates, federal spending, home ownership rates, house prices, educational attainment, voting paterns, employment statistics, and migration rates. 

In [24]:
clu = rdata[['CountyCode']]
data = pd.concat([Y, D, Z, clu], axis=1)

In [26]:
data.to_csv( r"../../../data/gun_clean2.csv" , index = False )

## The effect of gun ownership

### OLS

After preprocessing the data, we first look at simple regression of $Y_{j,t}$ on $D_{j,t-1}$ without controls as a baseline model.

In [27]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

In [ ]:
# # Run this line to avoid all the lines of code above
# data = pd.read_csv( r"../data/gun_clean2.csv"  )

In [28]:
# OLS clustering at the County level
model = "logghomr ~ logfssl"
baseline_ols = smf.ols(model , data=data).fit().get_robustcov_results(cov_type = "cluster", groups= data['CountyCode'])
baseline_ols_table = baseline_ols.summary2().tables[1]
print( baseline_ols_table.iloc[ 1 , 4:] )
baseline_ols_table.iloc[1, :]

[0.025    0.154480
0.975]    0.410129
Name: logfssl, dtype: float64


Coef.       0.282304
Std.Err.    0.064811
t           4.355825
P>|t|       0.000021
[0.025      0.154480
0.975]      0.410129
Name: logfssl, dtype: float64

The point estimate is $0.282$ with the confidence interval ranging from 0.155 to 0.41. This
suggests that increases in gun ownership rates are related to gun homicide rates - if gun ownership increases by 1% relative
to a trend then the predicted gun homicide rate goes up by 0.28%, without controlling for counties' characteristics.

Since our goal is to estimate the effect of gun ownership after controlling for a rich set county characteristics we next include the controls. First, we estimate the model by ols and then by an array of the modern regression methods using the double machine learning approach.

In [29]:
# define the variables
y = 'logghomr'

data_columns = list(data)
no_relev_col = ['logfssl', 'CountyCode', 'logghomr']

# This gives us: new_list = ['carrot' , 'lemon']
z = [col for col in data_columns if col not in no_relev_col]

In [30]:
control_formula = "logghomr" + ' ~ ' + 'logfssl + ' + ' + '.join( Z.columns.to_list() )


In [31]:
control_ols = smf.ols( control_formula , data=data).fit().get_robustcov_results(cov_type = "cluster", groups= data['CountyCode'])
control_ols_table = control_ols.summary2().tables[1]
print( control_ols_table.iloc[ 1 , 4:] )
control_ols_table.iloc[1, :]

[0.025    0.087645
0.975]    0.296981
Name: logfssl, dtype: float64


C:\Users\Roberto\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 196, but rank is 17
  warnings.warn('covariance of constraints does not have full '


Coef.       0.192313
Std.Err.    0.053070
t           3.623759
P>|t|       0.000371
[0.025      0.087645
0.975]      0.296981
Name: logfssl, dtype: float64

In R, the coefficients of the bellow variables are `Null`. However, in python we got a very high value.

In [32]:
control_ols_table.loc[['PPQ110D', 'PPQ120D'], :]

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
PPQ110D,-9.934306e+11,1.149386e+12,-0.864314,0.388483,-3.260327e+12,1.273466e+12
PPQ120D,-5.640049e+11,1.571563e+12,-0.358881,0.720074,-3.663548e+12,2.535538e+12


After controlling for a rich set of characteristics, the point estimate of gun ownership reduces to $0.19$.

# DML algorithm

Here we perform inference of the predictive coefficient $\beta$ in our partially linear statistical model, 

$$
Y = D\beta + g(Z) + \epsilon, \quad E (\epsilon | D, Z) = 0,
$$

using the **double machine learning** approach. 

For $\tilde Y = Y- E(Y|Z)$ and $\tilde D= D- E(D|Z)$, we can write
$$
\tilde Y = \alpha \tilde D + \epsilon, \quad E (\epsilon |\tilde D) =0.
$$

Using cross-fitting, we employ modern regression methods
to build estimators $\hat \ell(Z)$ and $\hat m(Z)$ of $\ell(Z):=E(Y|Z)$ and $m(Z):=E(D|Z)$ to obtain the estimates of the residualized quantities:

$$
\tilde Y_i = Y_i  - \hat \ell (Z_i),   \quad \tilde D_i = D_i - \hat m(Z_i), \quad \text{ for each } i = 1,\dots,n.
$$

Finally, using ordinary least squares of $\tilde Y_i$ on $\tilde D_i$, we obtain the 
estimate of $\beta$.

The following algorithm comsumes $Y, D, Z$, and a machine learning method for learning the residuals $\tilde Y$ and $\tilde D$, where the residuals are obtained by cross-validation (cross-fitting). Then, it prints the estimated coefficient $\beta$ and the corresponding standard error from the final OLS regression.

In [33]:
def DML2_for_PLM(z, d, y, dreg, yreg, nfold, clu):
    
    # Num ob observations
    nobs = z.shape[0]
    
    # Define folds indices 
    list_1 = [*range(0, nfold, 1)]*nobs
    sample = np.random.choice(nobs,nobs, replace=False).tolist()
    foldid = [list_1[index] for index in sample]

    # Create split function(similar to R)
    def split(x, f):
        count = max(f) + 1
        return tuple( list(itertools.compress(x, (el == i for el in f))) for i in range(count) ) 

    # Split observation indices into folds 
    list_2 = [*range(0, nobs, 1)]
    I = split(list_2, foldid)
    
    # Create array to save errors 
    dtil = np.zeros( len(z) ).reshape( len(z) , 1 )
    ytil = np.zeros( len(z) ).reshape( len(z) , 1 )
    
    # loop to save results
    for b in range(0,len(I)):
    
        # Split data - index to keep are in mask as booleans
        include_idx = set(I[b])  #Here should go I[b] Set is more efficient, but doesn't reorder your elements if that is desireable
        mask = np.array([(i in include_idx) for i in range(len(z))])

        # Lasso regression, excluding folds selected 
        dfit = dreg(z[~mask,], d[~mask,])
        yfit = yreg(z[~mask,], y[~mask,])

        # predict estimates using the 
        dhat = dfit.predict( z[mask,] )
        yhat = yfit.predict( z[mask,] )

        # save errors  
        dtil[mask] =  d[mask,] - dhat.reshape( len(I[b]) , 1 )
        ytil[mask] = y[mask,] - yhat.reshape( len(I[b]) , 1 )
        print(b, " ")
    
    # Create dataframe 
    data_2 = pd.DataFrame(np.concatenate( ( ytil, dtil,clu ), axis = 1), columns = ['ytil','dtil','CountyCode'])
   
    # OLS clustering at the County level
    model = "ytil ~ dtil"
    baseline_ols = smf.ols(model , data = data_2 ).fit().get_robustcov_results(cov_type = "cluster", groups= data_2['CountyCode'])
    coef_est = baseline_ols.summary2().tables[1]['Coef.']['dtil']
    se = baseline_ols.summary2().tables[1]['Std.Err.']['dtil']
    
    Final_result = { 'coef_est' : coef_est , 'se' : se , 'dtil' : dtil , 'ytil' : ytil }

    print("Coefficient is {}, SE is equal to {}".format(coef_est, se))
    
    return Final_result
    

Now, we apply the Double Machine Learning (DML) approach with different machine learning methods. First, we load the relevant libraries.

Let us, construct the input matrices.

In [34]:
# Create main variables
Y = data['logghomr']
D = data['logfssl']
Z = data.drop(['logghomr', 'logfssl', 'CountyCode'], axis=1)
CLU = data['CountyCode']

In [35]:
# as matrix
y = Y.to_numpy().reshape( len(Y) , 1 )
d = D.to_numpy().reshape( len(Y) , 1 )
z = Z.to_numpy()
clu = clu.to_numpy().reshape( len(Y) , 1 )

# Lasso 

In [36]:
def dreg(z,d):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(z, d)
    return result

def yreg(z,y):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(z, y)
    return result

DML2_lasso = DML2_for_PLM(z, d, y, dreg, yreg, 10, clu)

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.111e+01, tolerance: 4.886e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.504e+02, tolerance: 6.232e-02
  model = cd_fast.enet_coordinate_descent(


0  


C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.120e+01, tolerance: 4.840e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.446e+02, tolerance: 6.017e-02
  model = cd_fast.enet_coordinate_descent(


1  


C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.143e+01, tolerance: 4.963e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.588e+02, tolerance: 6.353e-02
  model = cd_fast.enet_coordinate_descent(


...

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.111e+01, tolerance: 4.864e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.436e+02, tolerance: 6.016e-02
  model = cd_fast.enet_coordinate_descent(


8  


C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.095e+01, tolerance: 4.806e-03
  model = cd_fast.enet_coordinate_descent(


9  
Coefficient is 0.21848801235033816, SE is equal to 0.05908567314112778


C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.543e+02, tolerance: 6.234e-02
  model = cd_fast.enet_coordinate_descent(


#### We use SelectFromModel to select variables which do not have a zero coefficient. It is done because we want to reduce dimensionality as rlasso( post = T ) does.

In [37]:
class Lasso_post:
    
    def __init__(self, alpha ):
        self.alpha = alpha

        
    def fit( self, X, Y ):
        self.X = X
        self.Y = Y
        lasso = linear_model.Lasso( alpha = self.alpha ).fit( X , Y )
        model = SelectFromModel( lasso , prefit = True )
        X_new = model.transform( X )
        # Gettin indices from columns which has variance for regression
        index_X = model.get_support()
        
        self.index = index_X
        new_x = X[ : ,  index_X ]
        
        lasso2 = linear_model.Lasso( alpha = self.alpha ).fit( new_x , Y )
        self.model = lasso2
        
        return self
    
    def predict( self , X ):
        
        dropped_X = X[ : , self.index ]
        
        predictions = self.model.predict( dropped_X )
        
        return predictions

Run the below code to verify whether Lasso_post functions as it is expected. 

In [38]:
# # setting alpha 
# alpha = 0.00000001

# # fit original model
# lasso = linear_model.Lasso( alpha = alpha ).fit( z , d )
# # select non zero coefficient variables
# model = SelectFromModel( lasso , prefit = True )
# # Dropping zero coefficient variables
# z_new = model.transform( z )
# # Getting the index of non zero coefficient variables
# index_X = model.get_support()
# # fitting a new model with the selected variables
# lasso2 = linear_model.Lasso( alpha = alpha ).fit( z[ : ,  index_X ] , d )
# # predictions with selected variables
# resultado = lasso2.predict( z[ : ,  index_X ] )

# # Making rlasso( post = T ) with Lasso_post 
# model1 = Lasso_post( alpha = alpha )
# mdfit = model1.fit( z, d )
# resultado2 = mdfit.predict( z )

# np.sum(resultado == resultado2)
# # the number of values which are equal are equal to the number of observations.
# # We conclude that our new class Lasso_post runs what we desire.

In [39]:
def dreg(z,d):
    alpha=0.00000001
    result = Lasso_post( alpha = alpha ).fit( z , d )
    return result

def yreg( z , y ):
    alpha = 0.00000001
    result = Lasso_post( alpha = alpha ).fit( z , y )
    return result

DML2_lasso_post = DML2_for_PLM(z, d, y, dreg, yreg, 10, clu)

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.149e+01, tolerance: 4.959e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.149e+01, tolerance: 4.959e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.524e+02, to

0  


C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.036e+01, tolerance: 4.737e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.036e+01, tolerance: 4.737e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.570e+02, to

1  


C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.012e+01, tolerance: 4.665e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.012e+01, tolerance: 4.665e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.542e+02, to

...

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.102e+01, tolerance: 4.907e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.102e+01, tolerance: 4.907e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.532e+02, to

8  


C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.099e+01, tolerance: 4.803e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.099e+01, tolerance: 4.803e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.506e+02, to

9  
Coefficient is 0.22682963710514406, SE is equal to 0.05543552034423103


C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.506e+02, tolerance: 6.190e-02
  model = cd_fast.enet_coordinate_descent(


### hdmy
We are going to replicate the above regressions but using `hmdpy` library.

In [40]:
import hdmpy
from statsmodels.tools import add_constant

In [41]:
class rlasso_sklearn:
    
    def __init__(self, post ):
        self.post = post
       
    def fit( self, X, Y ):
        
        self.X = X
        self.Y = Y
        
        # Standarization of X and Y
        self.rlasso_model = hdmpy.rlasso( X , Y , post = self.post )                
        return self
    
    def predict( self , X_1 ):
        self.X_1 = X_1
        beta = self.rlasso_model.est['coefficients'].to_numpy()
        
        if beta.sum() == 0:
            prediction = np.repeat( self.rlasso_model.est['intercept'] , self.X_1.shape[0] )
        
        else:
            prediction = ( add_constant( self.X_1 , has_constant = 'add') @ beta ).flatten()
                
        return prediction

In [42]:
# Post = false
def dreg(x, d):
    result = rlasso_sklearn( post = False ).fit( x , d )
    return result

def yreg(x,y):
    result = rlasso_sklearn( post = False ).fit( x , y )
    return result

DML2_lasso_hdmpy = DML2_for_PLM(z, d, y, dreg, yreg, 10, clu)

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is 0.22492680019879085, SE is equal to 0.05673777865627226


In [43]:
# Post = True
def dreg(x, d):
    result = rlasso_sklearn( post = True ).fit( x , d )
    return result

def yreg(x,y):
    result = rlasso_sklearn( post = True ).fit( x , y )
    return result

DML2_lasso_post_hdmpy = DML2_for_PLM(z, d, y, dreg, yreg, 10, clu)

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is 0.2227566208137165, SE is equal to 0.055508499271464014


We found out that the results are close. However, there are not a big differences between the Sklearn and the Hdmpy lasso function.

### cv.glmnet curiosities
1. According to [link1](https://stackoverflow.com/questions/24098212/what-does-the-option-normalize-true-in-lasso-sklearn-do) and [link2](https://statisticaloddsandends.wordpress.com/2018/11/15/a-deep-dive-into-glmnet-standardize/), It standardize **X** variables before estimation. In sklearn we have an option named **normalize**. It normalizes **X** by subtracting the mean and dividing by the l2-norm. Based on [link3](https://stackoverflow.com/questions/59846325/confusion-about-standardize-option-of-glmnet-package-in-r), **cv.glmnet (standardize = TRUE)** and **sklearn (normlize = True)** are not the same. We decided to use **StandardScaler** that meets suggestions made in **link3**. We proved this in the commented code below.

In [44]:
# a = np.array([ 6, 7, 8, 12, 15]).reshape( a.size, 1)

# scaler_a = StandardScaler()
# scaler_a.fit( a )
# print( scaler_a.transform( a ) )

# def std_1( x ):
#     val = ( (1 / x.size )*np.sum( ( x - np.mean( x ) ) ** 2 ) ) ** 0.5
#     return val

# ( a - np.mean(a) ) / std_1( a )

In [45]:
class standard_skl_model:
    
    def __init__(self, model ):
        self.model = model
       
    def fit( self, X, Y ):
        
        # Standarization of X and Y
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
                
        self.model.fit( std_X , Y )
                
        return self
    
    def predict( self , X ):
        
        self.scaler_X = StandardScaler()
        self.scaler_X.fit( X )
        std_X = self.scaler_X.transform( X )
        
        prediction = self.model.predict( std_X )
        
        return prediction

Run the below code to verify whether standard_skl_model functions as it is expected. 

In [46]:
# # Regular Operation using StandardScaler
# # Standarization of z
# scaler_z = StandardScaler()
# scaler_z.fit( z )
# std_z = scaler_z.transform( z )
# # Lasso CV prediction and getting normal 
# model1 = LassoCV(cv = 10 , random_state = 0 ).fit( std_z, std_d )
# predict1 = model1.predict( std_z )

# # Class standard_skl_model
# model2 = standard_skl_model( LassoCV(cv = 10 , random_state = 0 ) ).fit( z, d)
# predict2 = model2.predict( z )

# # Checking two results
# np.sum(real_predict1 == real_predict2)
# # the number of values which are equal are equal to the number of observations.
# # We conclude that our new class standard_skl_model runs what we desire.

In [47]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = standard_skl_model( LassoCV(cv = 10 , random_state = 0 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( LassoCV(cv = 10 , random_state = 0  ) ).fit( z, y )
    return result

DML2_lasso_cv = DML2_for_PLM(z, d, y, dreg, yreg, 2 , clu)

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0052421395178186, tolerance: 0.002468391302560743
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.007274296937804792, tolerance: 0.002468391302560743
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: Conv

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.1978519220234602, tolerance: 0.002468391302560743
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.5748640638509688, tolerance: 0.002468391302560743
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.901595321989644, tolerance: 0.002468391302560743
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.6681730052935411, tolerance: 0.0024326191429989248
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8246035442077577, tolerance: 0.0024326191429989248
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.962063760236008, tolerance: 0.0024326191429989248
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordin

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.2990175541379472, tolerance: 0.00246881027305081
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.4249729741852022, tolerance: 0.00246881027305081
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.43682097963980127, tolerance: 0.00246881027305081
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.9093249102864469, tolerance: 0.0024962940122308665
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.2146060297503638, tolerance: 0.0024962940122308665
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.4706670854807786, tolerance: 0.0024962940122308665
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordi

...

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26.837735336805338, tolerance: 0.03303359150490168
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26.058346279659958, tolerance: 0.03303359150490168
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 24.757722605516932, tolerance: 0.03303359150490168
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_d

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.139553455873596, tolerance: 0.03185301888082893
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.065398573715754, tolerance: 0.03185301888082893
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.883653328849476, tolerance: 0.03185301888082893
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_desc

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.1405917387803584, tolerance: 0.0324035321749889
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.5390581584549068, tolerance: 0.0324035321749889
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.224219270659944, tolerance: 0.0324035321749889
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_desce

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.27977369827340226, tolerance: 0.032938084117273544
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.2807723738040977, tolerance: 0.032938084117273544
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.4539704609616422, tolerance: 0.032938084117273544
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordina

1  
Coefficient is 0.21177086385364965, SE is equal to 0.05479059225746176


In [48]:
# DML with cross-validated Lasso:
def dreg(z,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.5, max_iter = 100000 ) ).fit( z, y )
    return result

DML2_elnet = DML2_for_PLM(z, d, y, dreg, yreg, 2 , clu)

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0  


C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.04854714999109433, tolerance: 0.03231204580476837
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective 

1  
Coefficient is 0.2040772144988563, SE is equal to 0.055850384952215615


In [49]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = standard_skl_model( ElasticNetCV( cv = 10 ,  random_state = 0 , l1_ratio = 0.0001 ) ).fit( z, d )
    return result

def yreg(z,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 , random_state = 0 , l1_ratio = 0.0001 ) ).fit( z, y )
    return result

DML2_ridge = DML2_for_PLM(z, d, y, dreg, yreg, 2, clu)

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0056518152667131005, tolerance: 0.0025298027975234408
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.010196076839477541, tolerance: 0.0025298027975234408
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633:

0  


C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.009604793834178338, tolerance: 0.002534244942959194
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.013019127137152253, tolerance: 0.002534244942959194
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: Co

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00920260076877355, tolerance: 0.0023365797490603767
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.019296302862798598, tolerance: 0.0023365797490603767
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.006188102235856974, tolerance: 0.0023365797490603767
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_c

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.008942376003403751, tolerance: 0.002552544111994602
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.016269317306544906, tolerance: 0.002552544111994602
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.023422094227971257, tolerance: 0.002552544111994602
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coo

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.04474953009646043, tolerance: 0.0025044136625464886
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1  
Coefficient is 0.18656560285852225, SE is equal to 0.05694954492241825


C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.149e-01, tolerance: 3.344e-02
  model = cd_fast.enet_coordinate_descent(


Here we also compute DML with OLS used as the ML method

In [50]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = LinearRegression().fit( z, d )
    return result

def yreg(z,y):
    result = LinearRegression().fit( z, y )
    return result

DML2_ols = DML2_for_PLM(z, d, y, dreg, yreg, 2, clu)

0  
1  
Coefficient is 0.16968371382318603, SE is equal to 0.05796033743026946


Next, we also apply Random Forest for comparison purposes.

### Random Forest


In [51]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [52]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
    return result

def yreg(z,y):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )
    return result

DML2_RF = DML2_for_PLM(z, d, y, dreg, yreg, 2, clu)   # set to 2 due to computation time

C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\719498043.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\719498043.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )


0  


C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\719498043.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\719498043.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 65 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )


1  
Coefficient is 0.15769986233640265, SE is equal to 0.05421048367045246


We conclude that the gun ownership rates are related to gun homicide rates - if gun ownership increases by 1% relative
to a trend then the predicted gun homicide rate goes up by about 0.20% controlling for counties' characteristics.

Finally, let's see which method is actually better. We compute RMSE for predicting D and Y, and see which
of the methods works better.


In [53]:
mods = [DML2_ols, DML2_lasso, DML2_lasso_post , DML2_lasso_cv, DML2_ridge, DML2_elnet, DML2_RF]
mods_name = ["DML2_ols", "DML2_lasso", "DML2_lasso_post" , "DML2_lasso_cv", 'DML2_ridge', 'DML2_elnet', 'DML2_RF']

def mdl( model , model_name ):
    
    RMSEY = np.sqrt( np.mean( model[ 'ytil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    RMSED = np.sqrt( np.mean( model[ 'dtil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    
    result = pd.DataFrame( { model_name : [ RMSEY , RMSED ]} , index = [ 'RMSEY' , 'RMSED' ])
    return result

RES = [ mdl( model , name ) for model, name in zip( mods , mods_name ) ]
    

pr_Res = pd.concat( RES, axis = 1)

pr_Res

,DML2_ols,DML2_lasso,DML2_lasso_post,DML2_lasso_cv,DML2_ridge,DML2_elnet,DML2_RF
RMSEY,0.002222,0.001122,0.000432,5.807320e-18,2.163512e-18,4.554761e-19,0.001034
RMSED,0.000991,0.000259,0.000380,2.704389e-19,7.686159e-19,3.558407e-19,0.000088


#### This verfies that the function DML2_for_PLM has no errors

In [54]:
np.where(DML2_lasso_post[ 'ytil' ] == 0)[0].size

0

It looks like the best method for predicting D is Lasso, and the best method for predicting Y is CV Ridge.

In [55]:
#DML with cross-validated Lasso:
def dreg(z,d):
    result = standard_skl_model( LassoCV(cv = 10 , random_state = 0 , alphas = [0]) ).fit( z, d )
    return result


def yreg(z,y):
    result = standard_skl_model( ElasticNetCV( cv = 10 ,  random_state = 0 , l1_ratio = 0.0001 ) ).fit( z, y )
    return result

DML2_best = DML2_for_PLM(z, d, y , dreg, yreg, 10, clu)

C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18.859739767368985, tolerance: 0.004404368303457231
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results 

0  


C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20.033052906147546, tolerance: 0.004617819216094837
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20.12733506749467, tolerance: 0.004688711734243341
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: UserWarning: Coordinate descent with alpha=0 may lead

1  


C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19.639322198639704, tolerance: 0.004530064895710938
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results 

...

7  


C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18.991259896816523, tolerance: 0.004421048074966
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16.979143058316886, tolerance: 0.00399655128875074
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: UserWarning: Coordinate descent with alpha=0 may lead to

8  


C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 17.63696081834954, tolerance: 0.004176034698172459
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Roberto\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16.818285

9  
Coefficient is 0.18475753592569952, SE is equal to 0.0532900157089008


In [56]:
table = np.zeros( ( 9 , 2 ))
table[ 0 , 0] = baseline_ols_table.iloc[ 1 , 0 ]
table[ 1 , 0] = control_ols_table.iloc[ 1 , 0 ]
table[ 2 , 0] = DML2_lasso['coef_est']
table[ 3 , 0] = DML2_lasso_post['coef_est']
table[ 4 , 0] = DML2_lasso_cv['coef_est']
table[ 5 , 0] = DML2_ridge['coef_est']
table[ 6 , 0] = DML2_elnet['coef_est']
table[ 7 , 0] = DML2_RF['coef_est']
table[ 8 , 0] = DML2_best['coef_est']
table[ 0 , 1] = baseline_ols_table.iloc[ 1 , 1 ]
table[ 1 , 1] = control_ols_table.iloc[ 1 , 1 ]
table[ 2 , 1] = DML2_lasso['se']
table[ 3 , 1] = DML2_lasso_post['se']
table[ 4 , 1] = DML2_lasso_cv['se']
table[ 5 , 1] = DML2_ridge['se']
table[ 6 , 1] = DML2_elnet['se']
table[ 7 , 1] = DML2_RF['se']
table[ 8 , 1] = DML2_best['se']

In [57]:
table = pd.DataFrame( table , index = [ "Baseline OLS", "Least Squares with controls", "Lasso", \
             "Post-Lasso", "CV Lasso","CV Elnet", "CV Ridge", "Random Forest", "Best" ] , \
            columns = ["Estimate","Standard Error"] )
table.round( 3 )

,Estimate,Standard Error
Baseline OLS,0.282,0.065
Least Squares with controls,0.192,0.053
Lasso,0.218,0.059
Post-Lasso,0.227,0.055
CV Lasso,0.212,0.055
CV Elnet,0.187,0.057
CV Ridge,0.204,0.056
Random Forest,0.158,0.054
Best,0.185,0.053


In [58]:
table.to_latex()

C:\Users\Roberto\AppData\Local\Temp\ipykernel_1308\1038981008.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  table.to_latex()


'\\begin{tabular}{lrr}\n\\toprule\n{} &  Estimate &  Standard Error \\\\\n\\midrule\nBaseline OLS                &  0.282304 &        0.064811 \\\\\nLeast Squares with controls &  0.192313 &        0.053070 \\\\\nLasso                       &  0.218488 &        0.059086 \\\\\nPost-Lasso                  &  0.226830 &        0.055436 \\\\\nCV Lasso                    &  0.211771 &        0.054791 \\\\\nCV Elnet                    &  0.186566 &        0.056950 \\\\\nCV Ridge                    &  0.204077 &        0.055850 \\\\\nRandom Forest               &  0.157700 &        0.054210 \\\\\nBest                        &  0.184758 &        0.053290 \\\\\n\\bottomrule\n\\end{tabular}\n'